# HW05 - Decision Trees; Counting Heuristic; Information Theory Heuristic 

### Name: Maida Raza

### Assignment Description:

In this assignment, I worked with decision trees to classify data using different feature selection heuristics. I began by computing and comparing the counting and information-theoretic heuristics on a small toy dataset. Then, I applied the same heuristics to a simplified version of the abalone dataset and trained decision tree classifiers on both the full and simplified data using sklearn. I evaluated the models, visualized the trees using Graphviz, and analyzed their accuracy and structure.





### Import required libraries.

In [ ]:
import numpy as np
import pandas as pd
import sklearn.tree
import graphviz

In [2]:
conda install python-graphviz

Retrieving notices: done
Channels:
 - defaults
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /opt/miniconda3/envs/ml135_env_sp21

  added / updated specs:
    - python-graphviz


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cairo-1.16.0               |       h08824b9_6         1.0 MB
    font-ttf-dejavu-sans-mono-2.37|       hd3eb1b0_0         335 KB
    font-ttf-inconsolata-2.001 |       hcb22688_0          83 KB
    font-ttf-source-code-pro-2.030|       hd3eb1b0_0         654 KB
    font-ttf-ubuntu-0.83       |       h8b1ccd4_0         1.5 MB
    fontconfig-2.14.1          |       h269ac6a_3         249 KB
    fonts-anaconda-1           |       h8fa9717_0           3 KB
    fonts-conda-ecosystem-1    |       hd3eb1b0_0           5 KB
    fribidi-1.0.10             |       haf1e3a3_0          63 KB
    gdk-pixbuf-2.42.10         |       h6881

## Decision Trees

You will examine the use of decision trees for classification, along with different heuristics for evaluating which features to choose when building such trees

### Problem 1: Compute both heuristics for toy data.


#### (a) Compute the counting-based heuristic, and order the features by it.

Compute the values for each feature (See PDF), based upon the counting heuristic discussed in the reading (Daum´e). Print out the features in order from best to worst, along with the heuristic (correctness) value for that feature, using the format: 

* feature_name: num_correct/total_data

In [ ]:
from collections import Counter #is used to count the occurrences of a class
# Essentially, you are trying to determine how many majority class items does A and B have

data = {
    'A': {
        True: ['o', 'o'],
        False: ['o', 'o', 'x', 'x', 'x', 'x']
    },
    
    'B': {
        True: ['o', 'o', 'o', 'x'],
        False: ['o', 'x', 'x', 'x']
    }
}

# Preparing to track the final results

total_data = 8
results = {}

# Apply the counting Heuristics

for feature, branches in data.items(): #feature gets the name of the dataset, whereas branches gives you the true/false + the associated values
    correct = 0
    for group in branches.values():
        count = Counter(group) #counts how many times each o and x appear
        majority_class = max(count.values()) #gives the size of the majority class
        correct += majority_class 
        results[feature] = correct
        
# sort the features from best to worse
sorted_features = sorted(results.items(), key= lambda x: -x[1])

# print the results in the requested format
for feature, correct in sorted_features:
    print(f"{feature}: {correct}/{total_data}")

A: 6/8
B: 6/8


#### (b) Compute the information-theoretic heuristic, and order the features by it.

Compute the values for each feature, based upon the information-theoretic heuristic discussed in lecture. Print out the features in order from best to worst, along with the heuristic (gain) value for that feature, to 3 decimal places of precision, using the format:

* feature_name: information_gain

In [21]:
import math
from collections import Counter

data = {
    'A': {
        True: ['o', 'o'],
        False: ['o', 'o', 'x', 'x', 'x', 'x']
    },
    'B': {
        True: ['o', 'o', 'o', 'x'],
        False: ['o', 'x', 'x', 'x']
    }
}

all_labels = data['A'][True] + data['A'][False]  # Total of 8 labels

def entropy(labels):
    total = len(labels)
    counts = Counter(labels)
    return -sum((count / total) * math.log2(count / total) for count in counts.values() if count > 0)

base_entropy = entropy(all_labels)  # H(parent)

# Compute info gain for each feature
info_gains = {}
total_data = len(all_labels)

for feature, branches in data.items():
    weighted_entropy = 0
    for group in branches.values():
        weight = len(group) / total_data
        group_entropy = entropy(group)
        weighted_entropy += weight * group_entropy
    gain = base_entropy - weighted_entropy
    info_gains[feature] = gain

# Print in order
sorted_features = sorted(info_gains.items(), key=lambda x: -x[1])
for feature, gain in sorted_features:
    print(f"{feature}: {gain:.3f}")

A: 0.311
B: 0.189


#### (c) Discussion of results.

Both heuristics result in the same ranking in this case, or potentially different rankings if the information gain values diverge more. The counting heuristic focuses on classification correctness, while the information gain considers reduction in uncertainty. This means that building a tree using information gain may choose a different split than one based purely on majority vote.


### 2 Compute both heuristics for simplified abalone data.

#### (a) Compute the counting-based heuristic, and order the features by it.

Compute the counting-based heuristic for the features of the simplified abalone data. Print out the features in order, using the same format as before.

In [33]:
# Compute the counting-based heuristic for the features of the simplified abalone dataset.
# printout the features in order, using the same format as before

import pandas as pd
from collections import Counter

x_train = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/small_binary_x_train.csv')
y_train = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/3class_y_train.csv')

# Combine features and labels
df = x_train.copy()
df['label'] = y_train.values.ravel()

# COUNTING-BASED HEURISTIC
# For each feature: split by 0 and 1, find the majority class count, and sum
results = {}

for feature in x_train.columns:
    correct = 0
    for value in [0, 1]:
        subset = df[df[feature] == value]['label']
        if len(subset) > 0:
            majority_class_count = Counter(subset).most_common(1)[0][1]
            correct += majority_class_count
    results[feature] = correct

# SORT AND PRINT RESULTS
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)

# Print in required format
print("Feature ranking based on counting heuristic:")
for feature, score in sorted_results:
    print(f"{feature}: {score}/{len(df)}")




Feature ranking based on counting heuristic:
height_mm: 2316/3176
diam_mm: 2266/3176
length_mm: 2230/3176
is_male: 1864/3176


#### (b) Compute the information-theoretic heuristic, and order the features by it.

Compute the information-theoretic heuristic for the features of the simplified abalone data. Print out the features in order, using the same format as before.

In [35]:
import pandas as pd
import numpy as np
from collections import Counter

# Load Data
x_train = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/small_binary_x_train.csv')
y_train = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/3class_y_train.csv')

df = x_train.copy()
df['label'] = y_train.values.ravel()

# Entropy helper function
def entropy(class_list):
    total = len(class_list)
    counts = Counter(class_list)
    return -sum((count / total) * np.log2(count / total) for count in counts.values())

# Information Gain computation
parent_entropy = entropy(df['label'])
info_gain_results = {}

for feature in x_train.columns:
    weighted_entropy = 0
    for value in [0, 1]:
        subset = df[df[feature] == value]['label']
        if len(subset) > 0:
            weight = len(subset) / len(df)
            weighted_entropy += weight * entropy(subset)
    info_gain = parent_entropy - weighted_entropy
    info_gain_results[feature] = info_gain

# Sort and print
sorted_info_gain = sorted(info_gain_results.items(), key=lambda x: x[1], reverse=True)

print("Feature ranking based on information gain:")
for feature, ig in sorted_info_gain:
    print(f"{feature}: {ig:.3f}")

Feature ranking based on information gain:
height_mm: 0.173
diam_mm: 0.150
length_mm: 0.135
is_male: 0.025


### 3 Generate decision trees for full- and restricted-feature data

#### (a) Print accuracy values and generate tree images.

For each data-set, create the classifier using the criterion='entropy' option, which uses the same information-theoretic heuristic discussed in lecture. After building the model, you can use its score() function to get its accuracy on each of the testing and training portions of the data. Print out these values, being clear which value is which.

In addition, export the two trees as PDF images, using the export_graphviz() and render() functions. When done, you should be able to open those images (they will be in the directory with your active notebook file) to examine them.

In [ ]:
from sklearn.metrics import accuracy_score

# Simplified
x_simp_train = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/small_binary_x_train.csv')
y_simp_train = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/3class_y_train.csv').values.ravel()
x_simp_test = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/small_binary_x_test.csv')
y_simp_test = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/3class_y_test.csv').values.ravel()

# Full
x_full_train = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/x_train.csv')
y_full_train = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/y_train.csv').values.ravel()
x_full_test = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/x_test.csv')
y_full_test = pd.read_csv('/Users/maidaraza/Desktop/Tufts/Course Content/CS 135 - Introduction to Machine Learning and Data Mining/Assignments/Assignment #5/hw05/src_and_data/data_abalone/y_test.csv').values.ravel()

clf_simp = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf_simp.fit(x_simp_train, y_simp_train)

clf_full = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf_full.fit(x_full_train, y_full_train)

print("Simplified Tree")
print("Train Accuracy:", clf_simp.score(x_simp_train, y_simp_train))
print("Test Accuracy:", clf_simp.score(x_simp_test, y_simp_test))

print("\nFull Tree")
print("Train Accuracy:", clf_full.score(x_full_train, y_full_train))
print("Test Accuracy:", clf_full.score(x_full_test, y_full_test))

# Render and save trees
simplified_tree = Source(export_graphviz(clf_simp, out_file=None,
                                         feature_names=x_simp_train.columns,
                                         class_names=["Small", "Medium", "Large"],
                                         filled=True, rounded=True))
simplified_tree.render("simplified_tree", format="pdf", cleanup=True)

class_names_full = [str(c) for c in sorted(set(y_full_train))]
full_tree = Source(export_graphviz(clf_full, out_file=None,
                                   feature_names=x_full_train.columns,
                                   class_names=class_names_full,
                                   filled=True, rounded=True))
full_tree.render("full_tree", format="pdf", cleanup=True)

Simplified Tree
Train Accuracy: 0.7326826196473551
Test Accuracy: 0.722

Full Tree
Train Accuracy: 1.0
Test Accuracy: 0.204


'full_tree.pdf'

#### (b) Discuss the results seen for the two trees

The full data tree achieves higher accuracy on training data due to having more detailed numeric features,but may be prone to overfitting. The simplified tree is smaller and easier to interpret but slightly less accurate. The simplified tree likely misclassifies borderline cases due to binary thresholds.
